## Setup

In [1]:
IN_COLAB = False
print("Running as a Jupyter notebook - intended for development only!")
from IPython import get_ipython

ipython = get_ipython()
# Code to automatically update the EasyTransformer code as its edited without restarting the kernel
ipython.magic("load_ext autoreload")
ipython.magic("autoreload 2")

# Import stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
from fancy_einsum import einsum
import tqdm.notebook as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader

from torchtyping import TensorType as TT
from typing import List, Union, Optional, Tuple
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML
from torchtyping import TensorType

import circuitsvis
from circuitsvis import attention

import easy_transformer
import easy_transformer.utils as utils
from easy_transformer.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from easy_transformer import EasyTransformer, EasyTransformerConfig, FactoredMatrix, ActivationCache

import pandas as pd
from tqdm import tqdm

torch.set_grad_enabled(False)



Running as a Jupyter notebook - intended for development only!


/tmp/ipykernel_21275/3157197756.py:7: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("load_ext autoreload")
/tmp/ipykernel_21275/3157197756.py:8: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("autoreload 2")


## Generate bigrams

In [2]:
model = EasyTransformer.from_pretrained(
    "NeelNanda/SoLU_1L512W_C4_Code",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    refactor_factored_attn_matrices=True,
    device="cpu"
)

attn_model = EasyTransformer.from_pretrained(
    "NeelNanda/Attn_Only_1L512W_C4_Code",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    refactor_factored_attn_matrices=True,
    device="cpu"
    )

tokenizer = model.tokenizer

Loading model: NeelNanda/SoLU_1L512W_C4_Code
Moving model to device:  cpu
Finished loading pretrained model NeelNanda/SoLU_1L512W_C4_Code into EasyTransformer!
Loading model: NeelNanda/Attn_Only_1L512W_C4_Code
Moving model to device:  cpu
Finished loading pretrained model NeelNanda/Attn_Only_1L512W_C4_Code into EasyTransformer!


In [6]:
tokens = []
# 48261
for tokenIndex in tqdm(range(48261)):
    # Get the token
    token = tokenizer.decode(tokenIndex)
    
    # Initialise the token details
    details = {
        "idx": tokenIndex,
        "token": token,
    }
    
    # Get the top k logits & corresponding tokens
    logits = model(token)[0][1]
    probs = F.softmax(logits, dim=-1)
    _, topKTokens = torch.topk(probs, 10, largest=True)
    
    for idx, token_idx in enumerate(topKTokens):
        token_idx = token_idx.item()
        token = tokenizer.decode(token_idx)
        prob = probs[token_idx]
        
        # Add to the details
        details[f"top_{idx}_idx"] = token_idx
        details[f"top_{idx}_token"] = token
        details[f"top_{idx}_prob"] = prob.item()
    
    # Add to tokens list
    tokens.append(details)
    
# Convert to a dataframe
bigrams = pd.DataFrame(tokens)

100%|██████████| 48261/48261 [09:49<00:00, 81.83it/s] 


In [13]:
bigrams.sort_values("top_0_prob", ascending=False).head(10)

,idx,token,top_0_idx,top_0_token,top_0_prob,top_1_idx,top_1_token,top_1_prob,top_2_idx,top_2_token,...,top_6_prob,top_7_idx,top_7_token,top_7_prob,top_8_idx,top_8_token,top_8_prob,top_9_idx,top_9_token,top_9_prob
17857,17857,pcbi,16,.,0.999555,533,which,0.000081,593,so,...,0.000013,345,he,0.000011,328,on,0.000009,276,in,0.000009
28707,28707,fasterxml,16,.,0.999089,28,:,0.000374,65,_,...,0.000024,25438,jackson,0.000020,15,-,0.000020,348,as,0.000020
28893,28893,umbre,619,ll,0.998891,252,on,0.000132,14392,lla,...,0.000032,68,b,0.000030,2139,ley,0.000028,1055,led,0.000028
46091,46091,HepG,20,2,0.995836,21,3,0.001424,19,1,...,0.000088,16,.,0.000079,15,-,0.000059,26,8,0.000054
28110,28110,surfact,1086,ants,0.995655,250,in,0.002054,386,ant,...,0.000078,834,ating,0.000074,422,ive,0.000045,312,ol,0.000042
30790,30790,courty,2130,ards,0.994725,12715,arded,0.001346,472,ard,...,0.000114,16,.,0.000113,680,ata,0.000074,1172,yl,0.000068
21543,21543,pione,398,ers,0.994663,255,er,0.001585,15,-,...,0.000089,675,erv,0.000089,258,en,0.000089,276,in,0.000079
39769,39769,googleapis,16,.,0.994287,65,_,0.001472,15034,\.,...,0.000288,41220,-%,0.000228,17,/,0.000198,3,!,0.000116
17490,17490,fibrobl,3153,astic,0.994141,1317,cells,0.000845,10,(,...,0.000100,19987,astically,0.000086,16,.,0.000077,13687,orer,0.000076
5817,5817,https,1333,://,0.993413,28,:,0.003046,16,.,...,0.000045,3,!,0.000043,15,-,0.000041,286,and,0.000040


In [8]:
bigrams["top_0_prob"].describe()

count    48261.000000
mean         0.193992
std          0.163758
min          0.005995
25%          0.091559
50%          0.143002
75%          0.227320
max          0.999555
Name: top_0_prob, dtype: float64

In [9]:
# Save to CSV
bigrams.to_csv("bigrams.csv", index=False)

### Generate prompts to test

In [24]:

prompts = []
answers_correct = []
answers_incorrect = []

i = 0

for _idx, row in bigrams.sort_values("top_0_prob", ascending=False).iterrows():
    # Get the token & most likely next tokens
    token = str(row["token"])
    most_likely = str(row["top_0_token"])
    less_likely = str(row["top_1_token"]) # Can change to 1-9 
    
    
    print(token, most_likely)
    
    # Create the prompt
    prompts.append(token)
    answers_correct.append(most_likely)
    answers_incorrect.append(less_likely)  
    
    i += 1
    if i >= 10:
        break  

pcbi .
fasterxml .
 umbre ll
 HepG 2
 surfact ants
 courty ards
 pione ers
googleapis .
 fibrobl astic
 https ://


## Attn vs SoLU

In [34]:
diffs = []
solu_res = []
attn_res = []

for idx, prompt in tqdm(enumerate(prompts)):
    answer = answers_correct[idx]
    answer_token = model.to_single_token(answer[0])

    solu_logits = model(prompt)
    attn_logits = attn_model(prompt)
    
    solu_last_logits = solu_logits[0][1]
    attn_last_logits = attn_logits[0][1]
    
    solu_probs = F.softmax(solu_last_logits, dim=-1)
    attn_probs = F.softmax(attn_last_logits, dim=-1)
    
    solu_correct_prob = solu_probs[answer_token]
    attn_correct_prob = attn_probs[answer_token]
    
    diff = solu_correct_prob - attn_correct_prob
    
    diffs.append(diff.item())
    solu_res.append(solu_correct_prob.item())
    attn_res.append(attn_correct_prob.item())

    
diffs_series = pd.Series(diffs)
solu_res_series = pd.Series(solu_res)
attn_res_series = pd.Series(attn_res)

# Show summary statistics
solu_res_series.describe()

10it [00:00, 49.89it/s]

torch.Size([1, 2, 48262])
torch.Size([1, 2, 48262])
torch.Size([1, 2, 48262])
torch.Size([1, 2, 48262])
torch.Size([1, 2, 48262])
torch.Size([1, 2, 48262])
torch.Size([1, 2, 48262])
torch.Size([1, 2, 48262])
torch.Size([1, 2, 48262])
torch.Size([1, 2, 48262])


count    1.000000e+01
mean     3.991859e-01
std      5.146846e-01
min      8.829575e-07
25%      9.193277e-06
50%      1.532977e-03
75%      9.954492e-01
max      9.995549e-01
dtype: float64